# Machine, Data & Learning

## Project | Genetic Algorithm

Team 30 | Tribrid
-----------
* Nitin Chandak (2019101024)
* Ayush Sharma (2019101004)

In [122]:
# Usefull Imports
import client as server
import numpy as np
import matplotlib.pyplot as plt
import random
import json
%matplotlib inline

### Initial Overfit Vector
```
[
    0.0, 
    -1.45799022e-12, 
    -2.28980078e-13,  
    4.62010753e-11, 
    -1.75214813e-10, 
    -1.83669770e-15,  
    8.52944060e-16,  
    2.29423303e-05, 
    -2.04721003e-06, 
    -1.59792834e-08,  
    9.98214034e-10
]
```

In [123]:
# Usefull Global Constants & lambda functions
TEAM_ID = 'colthAUIKUTfdh4qWrnHBhJzkyEm8kt4qIue1BKtyvLItfp8Po'
# INITIAL_OVERFIT_VECTOR = [
#     0.0, 
#     -1.45799022e-12, 
#     -2.28980078e-13,  
#     4.62010753e-11, 
#     -1.75214813e-10, 
#     -1.83669770e-15,  
#     8.52944060e-16,  
#     2.29423303e-05, 
#     -2.04721003e-06, 
#     -1.59792834e-08,  
#     9.98214034e-10
# ]
# INITIAL_OVERFIT_VECTOR=[ 0.00000000e+00, -1.38544571e-12, -2.39314288e-13,  4.62010753e-11,
#  -1.86809773e-10, -1.88180339e-15,  8.87303182e-16,  2.17511080e-05,
#  -2.02847485e-06, -1.59792834e-08,  9.94492532e-10,]
# INITIAL_OVERFIT_VECTOR=[ 0.00000000e+00, -1.34157007e-12, -2.43502736e-13,  4.18030722e-11,
#  -1.70243951e-10, -1.88180339e-15,  9.47209422e-16,  2.20840660e-05,
#  -2.11341874e-06, -1.59792834e-08,  9.01093380e-10,]
INITIAL_OVERFIT_VECTOR=[ 0.00000000e+00, -1.34157007e-12, -2.39500664e-13,  3.93964654e-11,
 -1.76372543e-10, -1.79528508e-15,  9.74197339e-16,  2.00010867e-05,
 -2.21274832e-06, -1.62774610e-08,  8.39523570e-10 ]
POPULATION_SIZE = 8
TOTAL_GENERATION = 25
MIN_GENE_VAL = -10
MAX_GENE_VAL = 10
MUTATION_PROBABILITY = 0.3
MUTATION_FACTOR = lambda : random.uniform(0.9, 1.1)
MATING_POOL_SIZE = lambda : random.randint(3,5)

In [124]:
def mutate_initial_individual(individual):
    '''
    This function will mutate every gene in an individual which is
    a vector of 11-D or array of 11 size with probability
    0.8 & leave other gene as it is.
    '''
    ret_individual = []
    for igene in range(len(individual)):
        ret_individual.append(individual[igene])
        if random.randint(1, 10) <= (10*0.8):
            final_gene = INITIAL_OVERFIT_VECTOR[igene]*MUTATION_FACTOR()
            if final_gene<MIN_GENE_VAL:
                ret_individual[igene] = MIN_GENE_VAL
            elif final_gene>MAX_GENE_VAL:
                ret_individual[igene] = MAX_GENE_VAL
            else:
                ret_individual[igene] = final_gene
    return ret_individual
            

def create_initial_population():
    '''
    Initializes the population. Gene is I's genenitial_Overfit_Vector's gene by default
    but replaced with MUTATION_FACTOR*Initial_Overfit_Vector
    with probability 0.8.
    Constraint on Gene is:
    MIN_GENE_VAL <= Gene <= MAX_GENE_VAL
    '''
    population = []
    for i in range(POPULATION_SIZE):
        individual = np.array(INITIAL_OVERFIT_VECTOR)
#         individual[individual != 0 ] = 0
        individual = np.array(mutate_initial_individual(individual))
        population.append(individual)
    return population

In [125]:
def population_fitness(population):
    curr_fitness=[]
    for individual in population:
        curr_fitness.append(individual_fitness(individual))
    return curr_fitness


def individual_fitness(individual):
    [train_error, validation_error]= server.get_errors(TEAM_ID,individual.tolist())
    print(train_error, validation_error)
    return  (1 / ((train_error + validation_error) + 2 * abs(train_error - validation_error)))

In [126]:
def sort_y_wrt_x(cx_list,cy_list):
    for i in range(POPULATION_SIZE):
        for j in range(i,POPULATION_SIZE):
            if cx_list[i] > cx_list[j]:
                cx_list[i],cx_list[j] = cx_list[j],cx_list[i]
                cy_list[i],cy_list[j] = cy_list[j],cy_list[i]
    return cx_list,cy_list

In [127]:
def cross_over(mating_population):
    mated_population = []
    index = len(mating_population)-1
    while index>0:
        I1 = mating_population[index]
        I2 = mating_population[index-1]
        crossover_point = random.randint(1, POPULATION_SIZE-1)
        C1 = list(I1[:crossover_point]) + list(I2[crossover_point:])
        C2 = list(I2[:crossover_point]) + list(I1[crossover_point:])
        mated_population.append(C1)
        mated_population.append(C2)
        index-=2
    return mated_population

In [128]:
def mutate_mated_population(mated_population):
    mutated_mated_population = []
    for individual in mated_population:
        temp_individual = []
        for igene in range(len(individual)):
            if random.randint(1, 10) <= (10*MUTATION_PROBABILITY):
                new_gene = individual[igene]*MUTATION_FACTOR()
                if new_gene<MIN_GENE_VAL:
                    temp_individual.append(MIN_GENE_VAL)
                elif new_gene>MAX_GENE_VAL:
                    temp_individual.append(MAX_GENE_VAL) 
                else:
                    temp_individual.append(new_gene)
            else:
                temp_individual.append(individual[igene])
        mutated_mated_population.append(temp_individual)
    return mutated_mated_population

In [129]:
def next_generation(mutated_mated_population,modified_population):
    next_gen_pop = mutated_mated_population[:]
    modified_pop = modified_population[::-1]
    remain_individual = POPULATION_SIZE-len(next_gen_pop)
    for i in range(remain_individual):
        next_gen_pop.append(modified_pop[i])
    return next_gen_pop

In [130]:
def reproduction_function(curr_fitness,curr_population):
    modified_fitness,modified_population = sort_y_wrt_x(list(curr_fitness),curr_population)
    mating_population = modified_population[(POPULATION_SIZE-MATING_POOL_SIZE()):]
    mated_population = cross_over(mating_population)
    mutated_mated_population = mutate_mated_population(mated_population)
    next_gen_population = next_generation(mutated_mated_population,modified_population)
    return next_gen_population

In [131]:
def get_best_vector(curr_fitness, curr_population):
    optimum_fitness_index = -1
    optimum_fitness = -1
    for ifitness in range(len(curr_fitness)):
        if curr_fitness[ifitness]>optimum_fitness:
            optimum_fitness_index = ifitness
    return curr_population[optimum_fitness_index],curr_fitness[optimum_fitness_index]

In [133]:
def run_algo():
    print('hi')
    curr_population = np.array(create_initial_population())
#     curr_fitness =  np.random.randint(1,101,POPULATION_SIZE)
    curr_fitness = np.array(population_fitness(curr_population))
    best_vector,best_vector_fitness = get_best_vector(curr_fitness, curr_population)
    curr_gen = 2
    while curr_gen<=TOTAL_GENERATION:
        print("cuurr_gen => ",curr_gen)
        print("cuurr_population => ",curr_population)
        curr_population = np.array(reproduction_function(curr_fitness,curr_population))
        curr_fitness = np.array(population_fitness(curr_population))
#         curr_fitness = np.random.randint(1,101,POPULATION_SIZE)
        temp_best_vector,temp_best_vector_fitness = get_best_vector(curr_fitness, curr_population)
        if temp_best_vector_fitness>best_vector_fitness:
            best_vector_fitness = temp_best_vector_fitness
            best_vector = temp_best_vector
        curr_gen+=1
    return best_vector,best_vector_fitness

In [134]:
print('hi')
best_vector,best_vector_fitness = run_algo()

hi
hi
42968895638026.734 75737080485606.19
13502210559967.38 22330837208938.742
94833023517315.16 173026142345075.03
21354336555407.285 35874301072054.74
16103912081413.771 27075819330110.79
34117250130627.07 59432066561201.56
30146916888975.617 52011886558354.36
30685199518324.543 52628684957195.42
cuurr_gen =>  2
cuurr_population =>  [[ 0.00000000e+00 -1.30815088e-12 -2.52624920e-13  3.81907287e-11
  -1.88304740e-10 -1.79528508e-15  1.00657118e-15  2.06652473e-05
  -2.21274832e-06 -1.70855128e-08  8.01465641e-10]
 [ 0.00000000e+00 -1.36323111e-12 -2.19187031e-13  3.61984834e-11
  -1.78502996e-10 -1.79528508e-15  8.88305404e-16  2.06938728e-05
  -2.09888117e-06 -1.71064367e-08  8.88626761e-10]
 [ 0.00000000e+00 -1.34157007e-12 -2.41446573e-13  3.98780489e-11
  -1.76372543e-10 -1.81831973e-15  9.25394698e-16  1.82383803e-05
  -2.43219951e-06 -1.76870577e-08  7.78957015e-10]
 [ 0.00000000e+00 -1.39830894e-12 -2.55866840e-13  3.68674349e-11
  -1.85992945e-10 -1.95304154e-15  1.06841449e-

31720674606046.895 54603710181833.25
30397390582213.53 52256840576518.68
30685199518324.543 52628684957195.42
30685199518324.543 52628684957195.42
30685199518324.543 52628684957195.42
30685199518324.543 52628684957195.42
30685199518324.543 52628684957195.42
30685199518324.543 52628684957195.42
cuurr_gen =>  7
cuurr_population =>  [[ 0.00000000e+00 -1.32224490e-12 -2.39354549e-13  4.00682289e-11
  -1.76372543e-10 -1.84826898e-15  9.51748802e-16  2.09568168e-05
  -2.21274832e-06 -1.49786697e-08  8.00970477e-10]
 [ 0.00000000e+00 -1.32224490e-12 -2.62445101e-13  4.04744794e-11
  -1.76372543e-10 -1.80060436e-15  9.40282521e-16  2.14682832e-05
  -2.21274832e-06 -1.49786697e-08  8.00970477e-10]
 [ 0.00000000e+00 -1.32224490e-12 -2.62445101e-13  4.00682289e-11
  -1.76372543e-10 -1.84826898e-15  9.51748802e-16  2.14682832e-05
  -2.21274832e-06 -1.49786697e-08  8.00970477e-10]
 [ 0.00000000e+00 -1.32224490e-12 -2.62445101e-13  4.00682289e-11
  -1.76372543e-10 -1.84826898e-15  9.51748802e-16  2.

24996385398166.777 42276998753457.98
26111111902136.547 44181540775160.05
30519665688360.49 52414873857121.08
17348212264063.793 28628641033526.03
30685199518324.543 52628684957195.42
30685199518324.543 52628684957195.42
30685199518324.543 52628684957195.42
30685199518324.543 52628684957195.42
cuurr_gen =>  12
cuurr_population =>  [[ 0.00000000e+00 -1.24226219e-12 -2.62445101e-13  4.00682289e-11
  -1.76372543e-10 -1.84826898e-15  1.01951965e-15  2.14682832e-05
  -2.15320786e-06 -1.49786697e-08  8.00970477e-10]
 [ 0.00000000e+00 -1.32224490e-12 -2.57528744e-13  4.00682289e-11
  -1.76372543e-10 -1.84826898e-15  9.51748802e-16  2.14682832e-05
  -2.21274832e-06 -1.39776470e-08  8.00970477e-10]
 [ 0.00000000e+00 -1.20848803e-12 -2.62445101e-13  3.75017167e-11
  -1.91674059e-10 -1.82088831e-15  9.60979587e-16  2.14682832e-05
  -2.21274832e-06 -1.49786697e-08  8.00970477e-10]
 [ 0.00000000e+00 -1.32224490e-12 -2.62445101e-13  4.00682289e-11
  -1.76372543e-10 -1.84826898e-15  9.85761369e-16  2

34583730714624.055 60068907008969.45
30685199518324.543 52628684957195.42
30685199518324.617 52628684957195.586
46533671657891.664 81453012109930.19
30685199518324.543 52628684957195.42
30685199518324.543 52628684957195.42
30685199518324.543 52628684957195.42
30685199518324.543 52628684957195.42
cuurr_gen =>  17
cuurr_population =>  [[ 0.00000000e+00 -1.24290220e-12 -2.62445101e-13  4.28641423e-11
  -1.91267092e-10 -1.84826898e-15  9.51748802e-16  1.95880621e-05
  -2.21274832e-06 -1.49786697e-08  8.00970477e-10]
 [ 0.00000000e+00 -1.32224490e-12 -2.62445101e-13  4.00682289e-11
  -1.76372543e-10 -1.84826898e-15  8.76139911e-16  2.14682832e-05
  -2.21274832e-06 -1.49786697e-08  8.00970477e-10]
 [ 0.00000000e+00 -1.32224490e-12 -2.59606064e-13  3.98402546e-11
  -1.76372543e-10 -1.84826898e-15  9.51748802e-16  2.14682832e-05
  -2.21274832e-06 -1.49786697e-08  8.00970477e-10]
 [ 0.00000000e+00 -1.32224490e-12 -2.66244269e-13  4.00682289e-11
  -1.76372543e-10 -1.84826898e-15  1.02561165e-15 

30275241241489.3 51846992708989.31
30685199518324.543 52628684957195.42
44878592429689.12 78103437581035.95
31393941586138.832 53542452039595.07
30685199518324.543 52628684957195.42
30685199518324.543 52628684957195.42
30685199518324.543 52628684957195.42
30685199518324.543 52628684957195.42
cuurr_gen =>  22
cuurr_population =>  [[ 0.00000000e+00 -1.32224490e-12 -2.62445101e-13  4.00682289e-11
  -1.76372543e-10 -1.84826898e-15  9.51748802e-16  2.16733740e-05
  -2.21274832e-06 -1.49786697e-08  8.00970477e-10]
 [ 0.00000000e+00 -1.34116408e-12 -2.62445101e-13  4.00682289e-11
  -1.76372543e-10 -1.84826898e-15  9.51748802e-16  2.14682832e-05
  -2.21274832e-06 -1.49786697e-08  8.00970477e-10]
 [ 0.00000000e+00 -1.40216844e-12 -2.62445101e-13  4.23354934e-11
  -1.76372543e-10 -1.84826898e-15  9.51748802e-16  2.14682832e-05
  -2.21274832e-06 -1.44766531e-08  7.36375251e-10]
 [ 0.00000000e+00 -1.32224490e-12 -2.62445101e-13  4.00682289e-11
  -1.81683523e-10 -1.96448875e-15  9.45794918e-16  2.1

In [135]:
print(best_vector,best_vector_fitness)
print(server.submit(TEAM_ID,best_vector.tolist()))
individual_fitness(best_vector)

[ 0.00000000e+00 -1.32224490e-12 -2.62445101e-13  4.00682289e-11
 -1.76372543e-10 -1.84826898e-15  9.51748802e-16  2.14682832e-05
 -2.21274832e-06 -1.49786697e-08  8.00970477e-10] 7.861582355109201e-15
successfully submitted
30685199518324.543 52628684957195.42


7.861582355109201e-15

In [121]:
# IOF=[ 0.00000000e+00, -1.34157007e-12, -2.43502736e-13,  4.18030722e-11,
#  -1.70243951e-10, -1.88180339e-15,  9.47209422e-16,  2.20840660e-05,
#  -2.11341874e-06, -1.59792834e-08,  9.01093380e-10]
# print(server.submit(TEAM_ID,IOF))